<a href="https://colab.research.google.com/github/Msg5513/HealthAl-Intelligent-Healthcare-Assistant-Using-IBM-Granite/blob/main/Health_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [ ]:

from transformers import pipeline

pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


AttributeError: 'NoneType' object has no attribute 'apply_chat_template'

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
# HealthAI -

print("Installing required packages...")
!pip install -q streamlit pandas numpy plotly requests python-dotenv pyngrok
!pip install -q scikit-learn matplotlib seaborn transformers accelerate

print("All packages installed successfully!")



print("Setting up ngrok for public access...")

NGROK_AUTH_TOKEN = "2z5CG6ebKCXAvR7RmXxYZ9CCIgC_7ZmzTMa6co7HKHngNiCZP"

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("Ngrok configured successfully!")



print("Creating HealthAI application file...")

healthai_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json
from typing import List, Dict
import logging
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class HealthAIModel:
    def __init__(self, pipeline):
        self.pipeline = pipeline

    def generate_response(self, prompt: str, max_tokens: int = 500) -> str:
        try:
            result = self.pipeline(
                prompt,
                max_new_tokens=max_tokens,
                temperature=0.7,
                do_sample=True,
                return_full_text=False
            )
            return result[0]['generated_text']
        except Exception as e:
            return f"Error generating response: {e}"

    def chat_response(self, user_message: str, chat_history: List = None) -> str:
        system_prompt = """You are HealthAI, a knowledgeable and empathetic healthcare assistant.
Provide accurate, helpful medical information while being clear about limitations.
Always recommend consulting healthcare professionals for serious concerns."""
        prompt = f"{system_prompt}\\n\\nUser Question: {user_message}\\n\\nHealthAI Response:"
        return self.generate_response(prompt, max_tokens=400)

    def predict_disease(self, symptoms: str, patient_data: Dict) -> str:
        prompt = f"""As a medical AI assistant, analyze the following symptoms and patient information.\\nPatient Info:\\n- Age: {patient_data.get('age')}\\n- Gender: {patient_data.get('gender')}\\n- Medical History: {patient_data.get('medical_history')}\\n\\nReported Symptoms: {symptoms}\\n\\nProvide:\\n1. Top 3 potential conditions with likelihood\\n2. Immediate recommendations\\n3. When to seek medical attention\\n4. Important disclaimers"""
        return self.generate_response(prompt, max_tokens=600)

    def generate_treatment_plan(self, condition: str, patient_data: Dict) -> str:
        prompt = f"""Create a treatment plan for:\\nCondition: {condition}\\nPatient Profile:\\n- Age: {patient_data.get('age')}\\n- Gender: {patient_data.get('gender')}\\n- Medical History: {patient_data.get('medical_history')}\\n- Current Medications: {patient_data.get('medications')}\\n- Allergies: {patient_data.get('allergies')}\\n\\nInclude:\\n1. Treatment objectives\\n2. Medication categories\\n3. Lifestyle advice\\n4. Follow-up suggestions\\n5. Warnings"""
        return self.generate_response(prompt, max_tokens=700)

    def analyze_health_trends(self, health_data: pd.DataFrame) -> str:
        if health_data.empty:
            return "No data provided."

        latest = health_data.iloc[-1]
        trends = {col: {
            "current": latest[col],
            "mean": round(health_data[col].mean(), 2)
        } for col in health_data.columns if col != "date"}

        prompt = f"""Analyze the following health metrics and provide advice:\\n\\n{json.dumps(trends, indent=2)}"""
        return self.generate_response(prompt, max_tokens=600)

@st.cache_resource

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
    model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct", torch_dtype="auto", device_map="auto")
    gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return HealthAIModel(gen_pipeline)

st.set_page_config(page_title="HealthAI Assistant", page_icon="🏥", layout="wide")

model = load_model()

st.title("🏥 HealthAI Assistant")

page = st.sidebar.selectbox("Navigation", [
    "🏠 Home",
    "💬 Chat with HealthAI",
    "🔎 Symptom Checker",
    "📋 Treatment Plans",
    "📊 Health Analytics",
    "📈 Health Metrics Tracker"
])

if page == "🏠 Home":
    st.markdown("Welcome to HealthAI. Use the sidebar to explore features.")

elif page == "💬 Chat with HealthAI":
    if "messages" not in st.session_state:
        st.session_state.messages = []
    for msg in st.session_state.messages:
        with st.chat_message(msg["role"]):
            st.markdown(msg["content"])
    if prompt := st.chat_input("Ask a health question..."):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)
        with st.chat_message("assistant"):
            response = model.chat_response(prompt)
            st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})

elif page == "🔎 Symptom Checker":
    age = st.number_input("Age", 1, 120, 30)
    gender = st.selectbox("Gender", ["Male", "Female", "Other"])
    medical_history = st.text_area("Medical History")
    symptoms = st.text_area("Describe your symptoms")
    if st.button("Analyze"):
        patient_data = {"age": age, "gender": gender, "medical_history": medical_history}
        with st.spinner("Analyzing..."):
            analysis = model.predict_disease(symptoms, patient_data)
            st.markdown(analysis)

elif page == "📋 Treatment Plans":
    age = st.number_input("Age", 1, 120, 30)
    gender = st.selectbox("Gender", ["Male", "Female", "Other"])
    condition = st.text_input("Condition")
    history = st.text_area("Medical History")
    meds = st.text_area("Current Medications")
    allergies = st.text_area("Allergies")
    if st.button("Generate Treatment Plan"):
        pdata = {"age": age, "gender": gender, "medical_history": history, "medications": meds, "allergies": allergies}
        with st.spinner("Generating plan..."):
            plan = model.generate_treatment_plan(condition, pdata)
            st.markdown(plan)

elif page == "📊 Health Analytics":
    if st.button("Generate Sample Data"):
        dates = pd.date_range("2024-01-01", "2024-12-31")
        df = pd.DataFrame({
            "date": dates,
            "heart_rate": np.random.normal(75, 10, len(dates)),
            "blood_pressure_sys": np.random.normal(120, 15, len(dates)),
            "blood_pressure_dia": np.random.normal(80, 10, len(dates)),
            "blood_glucose": np.random.normal(100, 20, len(dates))
        })
        st.session_state.health_data = df
    if "health_data" in st.session_state:
        df = st.session_state.health_data
        st.plotly_chart(px.line(df, x="date", y="heart_rate", title="Heart Rate"))
        st.plotly_chart(px.line(df, x="date", y=["blood_pressure_sys", "blood_pressure_dia"], title="Blood Pressure"))
        st.plotly_chart(px.line(df, x="date", y="blood_glucose", title="Blood Glucose"))
        if st.button("AI Health Insights"):
            with st.spinner("Analyzing..."):
                insight = model.analyze_health_trends(df)
                st.markdown(insight)

elif page == "📈 Health Metrics Tracker":
    with st.form("tracker"):
        hr = st.number_input("Heart Rate", 40, 200, 75)
        sys_bp = st.number_input("Systolic BP", 80, 250, 120)
        dia_bp = st.number_input("Diastolic BP", 40, 150, 80)
        glucose = st.number_input("Glucose", 50, 400, 100)
        submitted = st.form_submit_button("Save")
        if submitted:
            if "daily_metrics" not in st.session_state:
                st.session_state.daily_metrics = []
            st.session_state.daily_metrics.append({
                "date": datetime.now().strftime('%Y-%m-%d'),
                "heart_rate": hr,
                "sys_bp": sys_bp,
                "dia_bp": dia_bp,
                "glucose": glucose
            })
            st.success("Metrics saved!")
    if "daily_metrics" in st.session_state:
        st.dataframe(pd.DataFrame(st.session_state.daily_metrics))
'''

with open("healthai_app.py", "w", encoding='utf-8') as f:
    f.write(healthai_code)

print("HealthAI application file created successfully!")



import subprocess
import threading
import time

def run_streamlit():
    subprocess.run([
        'streamlit', 'run', 'healthai_app.py',
        '--server.port', '8501',
        '--server.headless', 'true',
        '--server.enableCORS', 'false',
        '--server.enableXsrfProtection', 'false'
    ])

print("Starting HealthAI application...")
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()

print("Waiting for Streamlit to initialize...")
time.sleep(15)

try:
    print("Creating public tunnel...")
    public_url = ngrok.connect(8501)
    print(f"\nAccess your HealthAI app at: {public_url}")
    print("\nApplication is running. Press Ctrl+C to stop.")
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nShutting down...")
    ngrok.kill()

Installing required packages...
All packages installed successfully!
Setting up ngrok for public access...
Ngrok configured successfully!
Creating HealthAI application file...
HealthAI application file created successfully!
Starting HealthAI application...
Waiting for Streamlit to initialize...
Creating public tunnel...

Access your HealthAI app at: NgrokTunnel: "https://9951-34-142-254-138.ngrok-free.app" -> "http://localhost:8501"

Application is running. Press Ctrl+C to stop.
